### LLM OpenAeroStruct

#### A multiagent tool to write optimization code for OpenAeroStruct with simply high level inputs from the user, i.e. I want to design a sweeped rectangular wing with elliptical lift distribution

In [ ]:
# Import the generative ai library
import google.generativeai as genai #type: ignore

# Import all the modules necessary to run OpenAeroStruct
import re
import time
import os
import subprocess
import warnings
import numpy as np
import pandas as pd #type: ignore
import openmdao.api as om
import json

# import OpenAeroStruct modules
from openaerostruct.geometry.utils import generate_mesh  # helper functions to generate mesh
from openaerostruct.geometry.geometry_group import Geometry
from openaerostruct.aerodynamics.aero_groups import AeroPoint

# Import the plotting libraries
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import niceplots  # Optional but recommended

#Ignore warnings and use the nice plots style
warnings.filterwarnings("ignore")

plt.style.use(
    niceplots.get_style("james-dark")
)  # Options: "doumont-light", "doumont-dark", "james-light", "james-dark"

In [ ]:
#Import the relevant agents
from Agents import ReformulatorAgent
from Agents import BaseMeshAgent
from Agents import GeometryAgent
from Agents import OptimizerAgent
from Agents import ResultsReaderAgent
from Agents import ReportWriter

In [ ]:
# Define the initial sample query from the user.
User_Request  = """For this design, we will keep the area constant at S = 400 m2. The span is b = 60 m. The cruise condition corresponds to CL = 0.5. Your job is to minimize drag at the condition of CL = 0.5 and you have complete freedom in the taper, twist, and sweep of the wing. Make a plot of the elliptical lift distribution."""

"""HERE ARE SOME BACKUP PROMPTS"""
#User_Request  = """For this design, we will keep the area constant at S = 400 m2. The span is b = 60 m. The cruise condition corresponds to CL = 0.5. Your job is to minimize drag at the condition of CL = 0.5 and you have complete freedom in the taper and sweep of the wing. Make a plot of the elliptical lift distribution."""
#User_Request  = """For this design, we will keep the area constant at S = 400 m2. The span is b = 60 m. The cruise condition corresponds to CL = 0.5. Your job is to minimize drag at the condition of CL = 0.5 and you have complete freedom in the taper, twist, and sweep of the wing. Make a plot of the elliptical lift distribution."""
#User_Request  = """For this design, we will keep the area at S = 100 m2. The span is b = 10 m. The cruise condition corresponds to CL = 2.0. Your job is to minimize drag at the condition of CL = 2.0 and you have complete freedom in the taper, twist, and sweep of the wing. Make a plot of the elliptical lift distribution."""

reformulator = ReformulatorAgent()
reformulator_output = reformulator.execute_task(User_Request)

In [ ]:
print("Reformulator Output:")
print(reformulator_output)

#### Now start writing the file into RunOAS to prepare for the optimization and plotting script using LLM.

In [ ]:
#Write the mesh prompt

MeshPrompt  = f"""For this wing desgin, the geometric parameters are as follows: {reformulator_output["geometric_constraint"]}, and the type of the wing mesh should be: {reformulator_output["baseline_wing_mesh"]}"""

mesher = BaseMeshAgent()
mesher_output = mesher.execute_task(MeshPrompt)

In [ ]:
print(mesher_output["python_code"])
print(mesher_output)

In [ ]:
#Geometry Setup
GeometryPrompt  = f"""For this wing desgin, we are allowed to change the following parameters: {reformulator_output["design_variables"]}"""

geometry_setup = GeometryAgent()
geometry_output = geometry_setup.execute_task(GeometryPrompt)

In [ ]:
print(geometry_output["python_code"])
print(geometry_output)

In [ ]:
# Optimization Setup
OptimizerPrompt  = f"""For this wing desgin, the optimization parameters are as follows: {reformulator_output["design_variables"]}, the objective function is {reformulator_output["objective_function"]}, the geometric constraints are {reformulator_output["geometric_constraint"]}, the flight condition is {reformulator_output["trim_condition"]}, and the optimization algorithm is {reformulator_output["optimization_algorithm"]}"""

optimizer_setup = OptimizerAgent()
optimizer_output = optimizer_setup.execute_task(OptimizerPrompt)

In [ ]:
print(optimizer_output["python_code"])
print(optimizer_output)

In [ ]:
#Write the chunks of code into the template file and run it
template_file = "/Users/conan/Desktop/LLM_Aerospace_Research/LLM_OpenAeroStruct/RunOAS_template.py"

#It should then find """Part 1: PUT THE BASELINE MESH OF THE WING HERE""" then add print("hi") after it
with open(template_file, "r") as file:
    template_code = file.read()
    
    # Replace all placeholders with the generated code
    template_code = template_code.replace(
        '"""Part 1: PUT THE BASELINE MESH OF THE WING HERE"""',
        f'"""Part 1: PUT THE BASELINE MESH OF THE WING HERE"""\n{mesher_output["python_code"]}'
    )
    template_code = template_code.replace(
        '"""Part 2:  DO THE GEOMETRY SETUP HERE"""',
        f'"""Part 2:  DO THE GEOMETRY SETUP HERE"""\n{geometry_output["python_code"]}'
    )
    template_code = template_code.replace(
        '"""Part 3: PUT THE OPTIMIZER HERE """',
        f'"""Part 3: PUT THE OPTIMIZER HERE """\n{optimizer_output["python_code"]}'
    )

# Write the modified code to a new file instead of modifying the template
run_oas_file = "/Users/conan/Desktop/LLM_Aerospace_Research/LLM_OpenAeroStruct/RunOAS.py"
with open(run_oas_file, "w") as file:
    file.write(template_code)

print("Created RunOAS.py file with all generated code sections")

In [ ]:
# Now run the script using a subprocess, also capture all the outputs, and save it as a text file.
output_file = "/Users/conan/Desktop/LLM_Aerospace_Research/LLM_OpenAeroStruct/output.txt"
with open(output_file, "w") as file:
    # Run the script and capture the output
    process = subprocess.Popen(["python3", run_oas_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    # Decode the output and write it to the file
    file.write(stdout.decode())
    file.write(stderr.decode())

### After Running the Optimization, the LLM should then have access to the plots then analyze the results

In [ ]:
HTML_Report = "/Users/conan/Desktop/LLM_Aerospace_Research/LLM_OpenAeroStruct/RunOAS_out/reports/opt_report.html"
pdf_output_path = "/Users/conan/Desktop/LLM_Aerospace_Research/LLM_OpenAeroStruct/Figures/Opt_History.pdf"

# Convert HTML report to PDF using pandoc
try:
    # Check if the HTML file exists
    if not os.path.exists(HTML_Report):
        print(f"Error: HTML report not found at {HTML_Report}")
    else:
        print("Converting HTML report to PDF...")
        # Use subprocess to call pandoc for the conversion
        result = subprocess.run(
            [
                "pandoc", HTML_Report,
                "-o", pdf_output_path,
                "--pdf-engine=/Library/TeX/texbin/pdflatex",
                "-V", "geometry:landscape,a4paper",  # Landscape + specific paper size
                "-V", "geometry:margin=5mm",        # Reduce margins
                "-V", "geometry:includeheadfoot",    # Use full page area
                "--variable", "mainfont=Helvetica",  # More compact font
                "--variable", "fontsize=5pt"        # Smaller base font size
            ],
            capture_output=True,
            text=True
        )
        
        if result.returncode == 0:
            print(f"Successfully converted to PDF: {pdf_output_path}")
        else:
            print(f"Error converting to PDF: {result.stderr}")
except Exception as e:
    print(f"An error occurred: {str(e)}")

In [ ]:
def run_plot_wing(file_path):
    """
    Execute the plot_wing command on a specified aero.db file
    
    Args:
        file_path (str): Path to the aero.db file
    """
    if not os.path.exists(file_path):
        print(f"Error: File '{file_path}' does not exist.")
        return
        
    try:
        # Run plot_wing command and capture output
        result = subprocess.run(["plot_wing", file_path], 
                               capture_output=True, 
                               text=True, 
                               check=True)
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error code {e.returncode}")
        print(f"Error message: {e.stderr}")
    except FileNotFoundError:
        print("Error: 'plot_wing' command not found. Please ensure it's installed and in your PATH.")

# Example usage
file_path = "/Users/conan/Desktop/LLM_Aerospace_Research/LLM_OpenAeroStruct/RunOAS_out/aero.db"
run_plot_wing(file_path)

In [ ]:
#Analyze the results without RAG#
#Geometry Setup
ResultsPrompt  = f"""The initial problem by the user is: {User_Request}, the reformulated problem is: {reformulator_output}"""

results_setup = ResultsReaderAgent()
results_output = results_setup.execute_task(ResultsPrompt)

In [ ]:
results_output

In [ ]:
#Analyze the results without RAG#
#Geometry Setup
ReportPrompt  = f"""The initial problem by the user is: {User_Request}, the reformulated problem is: {reformulator_output}, the analysis by the LLM is {results_output}"""

report_setup = ReportWriter()
report_output = report_setup.execute_task(ReportPrompt)

In [ ]:
#Write the latex output into /Users/conan/Desktop/LLM_Aerospace_Research/LLM_OpenAeroStruct/Figures/Report.tex and replace the original file.

report_file = "/Users/conan/Desktop/LLM_Aerospace_Research/LLM_OpenAeroStruct/Figures/Report.tex"
with open(report_file, "w") as file:
    file.write(report_output["ReportText"])